# P5: DATA ANALYSIS: Final analysis

### Notebook Description

This notebook provides a final analysis of flight data and delays, this time exploring the impact of route and weather conditions on delays.

####
## Notebook Configuration

In [1]:
# import psycopg2
from psycopg2 import connect
import pandas as pd
# from sqlalchemy import create_engine
import matplotlib.pyplot as plt
# import numpy as np
# from scipy.stats import ttest_ind
# from scipy.stats import f_oneway

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Connecting to the database</p>

In [2]:
cnx = connect(user='postgres_user', password='coderslab', host='localhost', database='airlines')

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Creating the 'read_sql_table' method to extract data from the database</p>

In [3]:
def read_sql_table(table_name):
    query = f"SELECT * FROM {table_name}"
    cursor = cnx.cursor()
    cursor.execute(query)
    column_names = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    cursor.close()
    cnx.close()
    df = pd.DataFrame(results, columns=column_names)   
    return df

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Loading the DataFrame 'flight_df_02', saved as a .csv file in the previous notebook</p>

In [4]:
flight_df = pd.read_csv("/Users/Me/Desktop/Airports_CL/data/processed/flight_df_02.csv")

In [5]:
flight_df

,id,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,...,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,is_delayed,is_weekend,distance_agg,manufacture_year,manufacture_year_agg
0,0,1,1,2,9E,N931XJ,3290,10874,10397,600,...,NaN,NaN,NaN,NaN,NaN,False,False,"(500, 600]",2008,2007
1,1,1,1,2,OH,N723PS,5495,10874,11057,704,...,NaN,NaN,NaN,NaN,NaN,True,False,"(300, 400]",2004,2004
2,2,1,1,2,OH,N525EA,5416,10874,11057,1944,...,NaN,NaN,NaN,NaN,NaN,False,False,"(300, 400]",2004,2004
3,3,1,1,2,OH,N706PS,5426,10874,11057,1521,...,NaN,NaN,NaN,NaN,NaN,False,False,"(300, 400]",2004,2004
4,4,1,1,2,OH,N262PS,5440,10874,14100,756,...,NaN,NaN,NaN,NaN,NaN,False,False,"(300, 400]",2004,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6922919,9251553,12,30,1,MQ,N818AE,3744,12511,13930,1411,...,2.0,0.0,0.0,0.0,31.0,True,False,"(400, 500]",2002,2001
6922920,9251554,12,30,1,MQ,N821AE,4235,12511,11298,1615,...,NaN,NaN,NaN,NaN,NaN,False,False,"(300, 400]",2002,2001
6922921,9251556,12,31,2,MQ,N245NN,4040,12511,11298,745,...,224.0,129.0,0.0,0.0,0.0,True,False,"(300, 400]",2016,2016
6922922,9251557,12,31,2,MQ,N806AE,3744,12511,13930,1411,...,3.0,0.0,0.0,0.0,71.0,True,False,"(400, 500]",2001,2001


###
## Enriching The Dataset With Additional Information From The 'airport_list' Table

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Loading the 'airport_list' table form the database</p>

In [6]:
airport_list_df = read_sql_table('airport_list')
airport_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    97 non-null     int64 
 1   origin_airport_id     97 non-null     int64 
 2   display_airport_name  97 non-null     object
 3   origin_city_name      97 non-null     object
 4   name                  97 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.9+ KB


<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Checking for duplicate values in the 'origin_airport_id' column</p>

In [7]:
duplicates_check = airport_list_df['origin_airport_id'].duplicated()
duplicates_check.value_counts()

origin_airport_id
False    97
Name: count, dtype: int64

(no duplicates)

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>LEFT-joining the 'flight_df' and 'airport_list_df' DataFrames using the 'origin_airport_id' column</p>

In [8]:
flight_df = flight_df.merge(
    airport_list_df[['origin_airport_id', 'origin_city_name']],
    on='origin_airport_id',
    how='left'
)

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Performing another LEFT-join between 'flight_df' and 'airport_list_df' using the 'dest_airport_id' column</p>

In [9]:
# re-naming the 'city' column to 'origin_city_name'
airport_list_df = airport_list_df.rename(columns={'city': 'origin_city_name'})

# performing a LEFT-join
flight_df = pd.merge(flight_df, airport_list_df[['origin_airport_id', 'origin_city_name']],
                     left_on='dest_airport_id', right_on='origin_airport_id', how='left')

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Cleaning up after the LEFT-join</p>

In [10]:
# merging the origin_airport_id_x and origin_airport_id_y columns into a single origin_airport_id column
flight_df['origin_airport_id'] = flight_df['origin_airport_id_x'].fillna(flight_df['origin_airport_id_y'])

# removing the origin_airport_id_x and origin_airport_id_y columns
flight_df.drop(['origin_airport_id_x', 'origin_airport_id_y'], axis=1, inplace=True)

# Renaming the origin_city_name_x and origin_city_name_y columns with proper names
flight_df.rename(columns={'origin_city_name_x': 'origin_city_name'}, inplace=True)
flight_df.rename(columns={'origin_city_name_y': 'destination_city_name'}, inplace=True)

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Final result</p>

In [11]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6922924 entries, 0 to 6922923
Data columns (total 35 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     int64  
 1   month                  int64  
 2   day_of_month           int64  
 3   day_of_week            int64  
 4   op_unique_carrier      object 
 5   tail_num               object 
 6   op_carrier_fl_num      int64  
 7   dest_airport_id        int64  
 8   crs_dep_time           int64  
 9   dep_time               float64
 10  dep_delay              float64
 11  dep_time_blk           object 
 12  crs_arr_time           int64  
 13  arr_time               float64
 14  arr_delay_new          float64
 15  arr_time_blk           object 
 16  cancelled              int64  
 17  crs_elapsed_time       float64
 18  actual_elapsed_time    float64
 19  distance               int64  
 20  distance_group         int64  
 21  year                   int64  
 22  carrier_delay     

###
## Further Analysis

<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Identifying the airports with the highest number of departures</p>

In [12]:
top_airports_origin_df = flight_df.groupby('origin_city_name').size().reset_index(name='count')
top_airports_origin_df = top_airports_origin_df.sort_values(by='count', ascending=False)
top_airports_origin_df

,origin_city_name,count
13,"Chicago, IL",408150
3,"Atlanta, GA",387620
18,"Dallas/Fort Worth, TX",294944
55,"New York, NY",257396
20,"Denver, CO",245160
...,...,...
52,"Minneapolis, MN",11528
68,"Portland, ME",10863
82,"Sanford, FL",10670
39,"Kona, HI",9268


<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Identifying the airports with the highest number of arrivals</p>

In [13]:
top_airports_destination_df = flight_df.groupby('destination_city_name').size().reset_index(name='count')
top_airports_destination_df = top_airports_destination_df.sort_values(by='count', ascending=False)
top_airports_destination_df

,destination_city_name,count
13,"Chicago, IL",407621
3,"Atlanta, GA",387542
18,"Dallas/Fort Worth, TX",294568
55,"New York, NY",257345
20,"Denver, CO",244898
...,...,...
52,"Minneapolis, MN",11575
68,"Portland, ME",10898
82,"Sanford, FL",10658
39,"Kona, HI",9277


<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Identifying the most frequently traveled route</p>

In [14]:
top_route_df = flight_df.groupby(['origin_city_name', 'destination_city_name']).size().reset_index(name='count')
top_route_df = top_route_df.sort_values(by='count', ascending=False)
top_route_df.head(1)

,origin_city_name,destination_city_name,count
483,"Chicago, IL","New York, NY",19126


<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Based on the assumption that a route has more than 500 flights, identifying the top 10 destinations with the least delays</p>

In [15]:
# grouping by origin and destination city and count the number of flights and delayed flights
route_delays_df = flight_df.groupby(['origin_city_name', 'destination_city_name']).agg(
    total_flights=('id', 'size'),
    delayed_flights=('is_delayed', 'sum')
).reset_index()

# filtering routes with more than 500 flights
route_delays_df = route_delays_df[route_delays_df['total_flights'] > 500]

# calculating the percentage of delayed flights for each route
route_delays_df['delay_percentage'] = route_delays_df['delayed_flights'] / route_delays_df['total_flights'] * 100

# sorting the DataFrame for the top 10 routes with the least delays
least_route_delays_df = route_delays_df.sort_values(by='delay_percentage', ascending=True).head(10)

least_route_delays_df

,origin_city_name,destination_city_name,total_flights,delayed_flights,delay_percentage
1443,"Kona, HI","Kahului, HI",1095,33,3.013699
262,"Boise, ID","Portland, OR",884,41,4.638009
2595,"Reno, NV","Portland, OR",501,25,4.990020
340,"Burbank, CA","Lihue, HI",6940,355,5.115274
1377,"Kahului, HI","Kona, HI",1094,58,5.301645
1442,"Kona, HI","Burbank, CA",7792,416,5.338809
2493,"Portland, OR","Fresno, CA",704,38,5.397727
1510,"Lihue, HI","Burbank, CA",6933,375,5.408914
338,"Burbank, CA","Kahului, HI",10725,584,5.445221
3161,"Tucson, AZ","Atlanta, GA",683,39,5.710102


<p style='background-color: #FFFFE0; margin-top:20px; padding:5px 15px; font-weight: 500'>Based on the assumption that a route has more than 500 flights, identifying the top 10 destinations with the most delays</p>

In [16]:
# grouping by origin and destination city and count the number of flights and delayed flights
route_delays_df = flight_df.groupby(['origin_city_name', 'destination_city_name']).agg(
    total_flights=('id', 'size'),
    delayed_flights=('is_delayed', 'sum')
).reset_index()

# filtering routes with more than 500 flights
route_delays_df = route_delays_df[route_delays_df['total_flights'] > 500]

# calculating the percentage of delayed flights for each route
route_delays_df['delay_percentage'] = route_delays_df['delayed_flights'] / route_delays_df['total_flights'] * 100

# sorting the DataFrame for the top 10 routes with the most delays
top_route_delays_df = route_delays_df.sort_values(by='delay_percentage', ascending=False).head(10)

top_route_delays_df

,origin_city_name,destination_city_name,total_flights,delayed_flights,delay_percentage
1262,"Houston, TX","Boston, MA",650,248,38.153846
2629,"Rochester, NY","Newark, NJ",649,245,37.750385
1897,"Myrtle Beach, SC","Newark, NJ",2542,900,35.405193
2099,"Norfolk, VA","Newark, NJ",736,260,35.326087
3068,"St. Louis, MO","Newark, NJ",1790,630,35.195531
2889,"San Juan, PR","Hartford, CT",652,227,34.815951
155,"Austin, TX","Newark, NJ",1688,581,34.419431
2882,"San Juan, PR","Boston, MA",1102,377,34.210526
2252,"Orlando, FL","Newark, NJ",6789,2276,33.524820
2403,"Phoenix, AZ","Newark, NJ",2535,845,33.333333


 # Wzbogacenie o dane pogodowe
 Używając procedury `read_sql_table`, wczytaj tabelę `airport_weather` do ramki `airport_weather_df`. Następnie wykonaj następujące polecenia:
 1. Pozostaw w ramce tylko następujące kolumny: `['station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd']`.
 1. Połącz ramki `airport_list_df` wraz z `airport_weather_df` po odpowiedniej kolumnie używając takiego złączenia, aby w wyniku usunąć te wiersze (lotniska), które nie posiadają danych pogodowych. Dodatkowo, upewnij się, że zostanie tylko dodana kolumna `origin_airport_id`.

 Tutaj wczytaj ramkę `airport_weather`

In [ ]:
connection = psycopg2.connect(
             host='localhost',
             user='postgres', 
             password='p',
             dbname='airlines'  
             )

In [ ]:
con = connect(host='localhost', database='airlines', user='postgres', password='p')
cursor = con.cursor()

In [ ]:
airport_weather_df = read_sql_table('airport_weather')
airport_weather_df.info()

 Tutaj oczyść ramkę `airport_weather_df` z nadmiarowych kolumn

In [ ]:
airport_weather_df = airport_weather_df[{'station', 'name', 'date', 'prcp', 'snow', 'snwd', 'tmax', 'awnd'}]
airport_weather_df.info()

Tutaj połącz ramki `airport_list_df` oraz `airport_weather_df` aktualizując `airport_weather_df`

In [ ]:
airport_weather_df = pd.merge(airport_weather_df, airport_list_df[['name','origin_airport_id']], on='name', how='inner')

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
airport_weather_df_expected_shape = (43394, 9)
airport_weather_df_shape = airport_weather_df.shape

assert airport_weather_df_expected_shape == airport_weather_df_shape, f'Nieodpowiedni wymiar ramki airport_weather_df, oczekiwano (wierszy, kolumn): {airport_weather_df_expected_shape}'

 ## Połączenie `airport_weather_df` oraz `flight_df`
 W celu złączenia ramek `airport_weather_df` oraz `flight_df` wykonaj następujące kroki:
 1. w ramce `aiport_weather_df` występuje kolumna `date`, zrzutuj ją na typ `DATETIME`.
 1. w ramce `flight_df` należy stworzyć nową kolumnę o nazwie `date`. W tym celu:
 	- złącz kolumny `month`, `day_of_month` oraz `year` razem, użyj następującego formatu daty: `YYYY-MM-DD`.
 	- zrzutuj kolumnę `date` na typ `DATETIME`.
 1. złącz ramki używając odpowiedniego klucza, wynik złączenia zapisz do ramki `flight_df`. Uzyj złącznia typu `LEFT JOIN`.

 > Dlaczego istotne jest zachowanie typów przy złączeniu?

W trakcie pracy możesz posłużyć się następującymi artykułami z `LMS`:
 - `Python - analiza danych > Dzień 6 - Pandas > Merge`
 - `Python - analiza danych > Dzień 6 - Pandas > Praca z datetime`
 - Dokumentacje metody `to_datetime`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
 - Dostępne formaty dat: [klik](https://www.programiz.com/python-programming/datetime/strftime) - sekcja `Format Code List`

 Tutaj zrzutuj kolumnę `date` na `DATETIME` w ramce `airport_weather_df`

In [ ]:
airport_weather_df['date'] = pd.to_datetime(airport_weather_df['date'])

 Tutaj stwórz kolumnę `date` w ramce `flight_df`. Pamiętaj, aby była ona również typu `DATETIME`

In [ ]:
flight_df['date'] = pd.to_datetime(flight_df['year'].astype(str) + '-' + flight_df['month'].astype(str) + '-' + flight_df['day_of_month'].astype(str))

In [ ]:
flight_df['date'] = pd.to_datetime(flight_df['date'])

 Tutaj złącz tabele `airport_weather_df` oraz `flight_df`

In [ ]:
# merged_df = flight_df.join(airport_weather_df, on='date', how='left', lsuffix='_flight', rsuffix='_weather')

In [ ]:
# merged_df = pd.merge(airport_weather_df, flight_df, on='origin_airport_id', how='left')

In [ ]:
# merged_df = pd.merge(airport_weather_df, flight_df, on='date', how='left')

In [ ]:
flight_df = flight_df.merge(airport_weather_df, on=['origin_airport_id', 'date'], how='left')

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_rows_amount = 6922924
assert flight_df.shape[0] == flight_df_expected_rows_amount, 'Ups, zmieniła się liczba wierszy...'

In [ ]:
flight_df.info()

In [ ]:
flight_df.to_csv("/Users/Me/Desktop/Airports_CL/data/processed/flight_df_03.csv", index=False)

 # Praca samodzielna
 Używając `flight_df` zbadaj następujące hipotezy:
 1. Opady śniegu w lotnisku wylotowym wpływają na **wielkość** opóźnienia lotów (kolumna `snow`).
 1. Wielkość pokrywy śnieżnej wpływa na **wielkość** opóźnienia lotów (kolumna `snwd`).
 1. Temperatura maksymalna wpływa na **wielkość** opóźnienia lotów (kolumna `tmax`).
 W każdym ćwiczeniu pamiętaj o uwzględnieniu tylko tych zjawisk atmosferycznych, które były zaobserwowane (`>`). Przy wykonywaniu tego zadania masz pełną dowolność.

> **Wskazówka:**  
> Pamiętaj o tym, aby każda analiza była porównywalna, tj. dokonana przy podobnych założeniach.

 ## Analiza dla kolumny `snow`

 ### Określenie statystyk opisowych dla kolumny `snow`

In [ ]:
flight_df['snow'].describe().round(decimals=2)

Korelacja

In [ ]:
correlation = flight_df['snow'].corr(flight_df['dep_delay'])
correlation

Ponieważ wynik jest bliski 0, można stwierdzić, że nie ma wystarczających dowodów na liniową zależność między opadami śniegu a opóźnieniami lotów. Oznacza to, że wielkość opóźnień lotów nie jest znacząco zmieniana przez poziom opadów śniegu w lotnisku wylotowym na podstawie dostępnych danych.
Nie jest to jednak logiczne, więc potrzebe są dodatkowe analizy.

Test t-studenta

In [ ]:
threshold = 5
low_snow_group = flight_df[flight_df['snow'] < threshold]['dep_delay']
high_snow_group = flight_df[flight_df['snow'] >= threshold]['dep_delay']

In [ ]:
t_statistic, p_value = ttest_ind(low_snow_group, high_snow_group)
print("Wartość statystyki t:", t_statistic)
print("Wartość p:", p_value)

Wartość statystyki t wynosząca -91.03941737145492 wskazuje na dużą różnicę między grupami opartymi na poziomach opadów śniegu. Wartość t jest ujemna, co sugeruje, że średnia wartość opóźnień lotów jest istotnie niższa w grupie z wyższymi opadami śniegu w porównaniu do grupy z niższymi opadami śniegu.
Innymi słowy - wynik testu t-studenta sugeruje, że opady śniegu w lotnisku wylotowym mają istotny wpływ na wielkość opóźnień lotów.

Statystyka ANOVA 

In [ ]:
threshold1 = 2  # Próg dla niskich opadów śniegu
threshold2 = 5  # Próg dla średnich opadów śniegu

# Załóżmy, że mamy trzy grupy: niska, średnia i wysoka opady śniegu
low_snow_group = flight_df[flight_df['snow'] < threshold1]['dep_delay']
medium_snow_group = flight_df[(flight_df['snow'] >= threshold1) & (flight_df['snow'] < threshold2)]['dep_delay']
high_snow_group = flight_df[flight_df['snow'] >= threshold2]['dep_delay']

f_statistic, p_value = f_oneway(low_snow_group, medium_snow_group, high_snow_group)
print("Wartość statystyki F:", f_statistic)
print("Wartość p:", p_value)

Statystyka ANOVA swoimi wynikami (wysoki poziom F oraz wartość p: 0) potwierdza hipoteze z testu t-studenta, że opady śniegu w lotnisku wylotowym mają istotny wpływ na wielkość opóźnień lotów.

 ## Analiza dla kolumny `snwd`

 ### Określenie statystyk opisowych dla kolumny `snwd`

In [ ]:
flight_df['snwd'].describe().round(decimals=2)

Korelacja

In [ ]:
correlation = flight_df['snwd'].corr(flight_df['dep_delay'])
correlation

Ponieważ wynik jest bliski 0, można stwierdzić, że nie ma wystarczających dowodów na liniową zależność między wielkością pokrywy śnieżnej a opóźnieniami lotów. Oznacza to, że wielkość opóźnień lotów nie jest znacząco zmieniana przez poziom opadów śniegu w lotnisku wylotowym na podstawie dostępnych danych.
Nie jest to jednak logiczne, więc potrzebe są dodatkowe analizy.

Test t-studenta

In [ ]:
threshold = 5
low_snwd_group = flight_df[flight_df['snwd'] < threshold]['dep_delay']
high_snwd_group = flight_df[flight_df['snwd'] >= threshold]['dep_delay']

In [ ]:
t_statistic, p_value = ttest_ind(low_snwd_group, high_snwd_group)
print("Wartość statystyki t:", t_statistic)
print("Wartość p:", p_value)

Wartość statystyki t wynosząca -48.746791464167075 wskazuje na dużą różnicę między grupami opartymi na poziomach wielkości pokrywy śnieżnej. Wartość t jest ujemna, co sugeruje, że średnia wartość opóźnień lotów jest istotnie niższa w grupie z większą ilością pokrywy śnieżnej w porównaniu do grupy lotnisk z mniejszą pokrywą śnieżną.
Innymi słowy - wynik testu t-studenta sugeruje, że wielkość pokrywy śnieżnej na lotnisku wylotowym ma istotny wpływ na wielkość opóźnień lotów.

Statystyka ANOVA 

In [ ]:
threshold1 = 2  # Próg dla niskich opadów śniegu
threshold2 = 5  # Próg dla średnich opadów śniegu

# Załóżmy, że mamy trzy grupy: niska, średnia i wysoka opady śniegu
low_snwd_group = flight_df[flight_df['snwd'] < threshold1]['dep_delay']
medium_snwd_group = flight_df[(flight_df['snwd'] >= threshold1) & (flight_df['snwd'] < threshold2)]['dep_delay']
high_snwd_group = flight_df[flight_df['snwd'] >= threshold2]['dep_delay']

f_statistic, p_value = f_oneway(low_snwd_group, medium_snwd_group, high_snwd_group)
print("Wartość statystyki F:", f_statistic)
print("Wartość p:", p_value)

Statystyka ANOVA swoimi wynikami (wysoki poziom F oraz wartość p: 0) potwierdza hipoteze z testu t-studenta, że wielkość pokrywy śnieżnej na lotnisku wylotowym ma istotny wpływ na wielkość opóźnień lotów.

 ## Analiza dla kolumny `tmax`

 ### Określenie statystyk opisowych dla kolumny `tmax`

In [ ]:
flight_df['tmax'].describe().round(decimals=2)

Korelacja

In [ ]:
correlation = flight_df['tmax'].corr(flight_df['dep_delay'])
correlation

Ponieważ wynik jest ujemny, można stwierdzić, że nie istnieje liniowa zależność między maksymalną temperaturą a opóźnieniami lotów. Jednakże jest on bardzo bliski 0, więc dla pewności, poniżej znajduje się dodatkowy test t-studenta

Test t-studenta

In [ ]:
threshold = 5
low_tmax_group = flight_df[flight_df['tmax'] < threshold]['dep_delay']
high_tmax_group = flight_df[flight_df['tmax'] >= threshold]['dep_delay']

In [ ]:
t_statistic, p_value = ttest_ind(low_tmax_group, high_tmax_group)
print("Wartość statystyki t:", t_statistic)
print("Wartość p:", p_value)

Na podstawie wyników testu t-studenta moża wyciagnąć wniosek, że maksymalna termeratura ma znaczący wpływ na opóźnienia lotów.

# Podsumowanie
W tej części warsztatu dokonaliśmy kompleksowej analizy posiadanego zbioru danych. Eksploracja
pozwoliła nam na zapoznanie się z cechami charakterystycznymi lotów - wiemy już, które 
zmienne mogą mieć wpływ na opóźnienia lotów, a które nie. Co warto podkreślić, skupiliśmy się na wielu
aspektach tej analizy, co otwiera potencjalnie również inne możliwości dalszej pracy nad tą bazą.

W tym momencie przejdziemy do kolejnego kroku, w którym, na podstawie tej analizy, przygotujemy 
system raportowy. Zanim jednak stworzymy dashboard, potrzebujemy zaktualizować naszą bazę danych.